In [1]:
from astropy.table import Table
import numpy as np
import mpmath as mm

from scipy.spatial import KDTree

from vast.voidfinder.distance import z_to_comoving_dist

In [2]:
vfv = Table.read('/home/codebind/Bootes/Holes.txt', format = 'ascii.commented_header')

In [3]:
D2R = np.pi/180

def radec_to_cart(ra,decl,dist):
    x = dist*np.cos(ra*D2R)*np.cos(decl*D2R)
    y = dist*np.sin(ra*D2R)*np.cos(decl*D2R)
    z = dist*np.sin(decl*D2R)
    return x,y,z    

In [4]:
bootes_z = 15500/300000
bra, bdecl, bdist = 222.5, 46, z_to_comoving_dist(np.array([bootes_z],dtype=np.float32),0.315,1)
bx,by,bz = radec_to_cart(bra, bdecl, bdist)

In [5]:
#centering all void finder voids on the bootes region
vfvb = Table()
vfvb['x'] = vfv['x']-bx
vfvb['y'] = vfv['y']-by
vfvb['z'] = vfv['z']-bz
vfvb['radius'] = vfv['radius']
vfvb['flag'] = vfv['flag']

In [6]:
def create_grid(prec):
    #creating a cube coordinate grid in the region of Bootes and putting them into a table
    xs = np.linspace(-62,62,int(124/prec))
    ys = np.linspace(-62,62,int(124/prec))
    zs = np.linspace(-62,62,int(124/prec))
    X, Y, Z = np.meshgrid(xs, ys, zs,sparse=True)
    x = X.reshape((np.prod(X.shape),))
    y = Y.reshape((np.prod(Y.shape),))
    z = Z.reshape((np.prod(Z.shape),))
    table = list(zip(x,y,z))
    
    coords = Table()
    coords['x'] = ()
    coords['y'] = ()
    coords['z'] = ()
    for i in range(len(table)):
        coords.add_row(table[i])
    
    #cutting off all points outside the spherical radius of Bootes Region
    bpoints = []
    for j in range(len(coords)):
        rad = mm.sqrt(coords['x'][j]**2+coords['y'][j]**2+coords['z'][j]**2)
        if (rad <= 62):
            bpoints.append(j)
    
    grid = coords[bpoints]
    grid['pnum'] = range(len(grid))
    return grid

In [7]:
vfgrid = create_grid(.01)

In [8]:
def VF_volume_search(grid, depth):
    #compiling all void points
    vfpoints = []
    
    #creating a KDTree to match gridpoints to the nearest void
    kdt = KDTree(np.array([vfvb['x'],vfvb['y'],vfvb['z']]).T)
    distance, index = kdt.query(np.array([grid['x'],grid['y'],grid['z']]).T, k = depth)
    grid['void_id'] = index
    grid['void_dis'] = distance
    
    for i in range(len(grid)):
        a = grid['void_id'][i]
        b = grid['void_dis'][i]
        for j,v in enumerate(a):
            if b[j] <= vfvb['radius'][v]:
                vfpoints.append(i)
                break

    vfvol = grid[vfpoints]
    return vfvol

In [9]:
vfvol = VF_volume_search(vfgrid, 25)
print(len(vfvol))
print(len(vfgrid))
print(len(vfvol)/len(vfgrid)) #percentage of Bootes region that is in the VF voids

5020
7158
0.7013132159821179


In [10]:
#start of V2 stuff

#V2 Files
gdata = Table.read("/home/codebind/Bootes/NSA_Gals.txt", format='ascii.commented_header')
gzdata = Table.read("/home/codebind/Bootes/DR7_galzones.dat",format='ascii.commented_header')
zvdata = Table.read("/home/codebind/Bootes/DR7_zonevoids.dat",format='ascii.commented_header')

In [11]:
gdata['zone'] = gzdata['zone']

In [12]:
def toCoord(r,ra,dec):
    c1 = r*np.cos(ra*D2R)*np.cos(dec*D2R)
    c2 = r*np.sin(ra*D2R)*np.cos(dec*D2R)
    c3 = r*np.sin(dec*D2R)
    return c1,c2,c3

In [13]:
v2grid = create_grid(.01)

In [14]:
def V2_volume_search(grid):
    #matching void ID to zones in the galaxy data 
    gz = gdata['zone']
    zv = zvdata['void0']
    gv = -1*np.ones(len(gz))
    for i,z in enumerate(gz):
        if z >-1:
            gv[i] = zv[z]

    gdata['void0'] = gv
    
    #creating a KDTree to match grid points to the nearest galaxy
    gr   = z_to_comoving_dist(np.array(gdata['redshift'],dtype=np.float32),0.315,1)
    gra  = gdata['ra']
    gdec = gdata['dec']
    
    gx,gy,gz = toCoord(gr,gra,gdec)
    kdt = KDTree(np.array([(gx-bx),(gy-by),(gz-bz)]).T)
    x = grid['x']
    y = grid['y']
    z = grid['z']
    distance, galaxy = kdt.query(np.array([x,y,z]).T, k =[1])
    grid['gal'] = galaxy
    
    #determining if grid point is in a void by the status of its nearest neighboring galaxy
    grid['void0'] = -1*np.ones(len(grid))
    for i,g in enumerate(grid['gal']):
        grid['void0'][i] = gdata['void0'][g]
        
    #compiling all void points (use the the commented lines instead of the table for a simple tally)
    v2points = []
    for x in range(len(grid)):
        if grid['void0'][x] >= 0:
            v2points.append(x)
    
    v2vol = grid[v2points]
    return v2vol

In [15]:
v2vol = V2_volume_search(v2grid)
print(len(v2vol))
print(len(v2grid))
print(len(v2vol)/len(v2grid)) #percentage of the Bootes region that is in the V2 voids

4742
7158
0.6624755518301202


In [16]:
def volume_match(vol1, vol2):
    tally = 0
    for num in vol1['pnum']:
        if num in vol2['pnum']:
            tally = tally + 1
    return tally

In [17]:
matchpoints = (volume_match(v2vol,vfvol))

In [18]:
vf_overlap = matchpoints/len(vfvol) #percentage of the VF volume that overlaps V2
v2_overlap = matchpoints/len(v2vol) #percentage of the V2 volume that overlaps VF
total_overlap = matchpoints/len(vfgrid) #percentage of the total Bootes Volume that is overlapped by both VF & V2

print(vf_overlap)
print(v2_overlap)
print(total_overlap)

0.6193227091633466
0.6556305356389709
0.4343392008941045
